In [33]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

from itertools import combinations

# apparently scipy has some bug showing an error
# https://github.com/scipy/scipy/issues/5998
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

# get the cleaned data
data = pd.read_csv('../yjkweon24/kc_house_cleaned.csv')
data.head()

,Unnamed: 0,id,date,price,bedrooms,bathrooms,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,lat,long,sqft_living15
0,1,7129300520,20141013,221900.0,3,1.00,1.0,0,0,3,7,1180,0,1955,47.5112,-122.257,1340
1,2,6414100192,20141209,538000.0,3,2.25,2.0,0,0,3,7,2170,400,1951,47.7210,-122.319,1690
2,3,5631500400,20150225,180000.0,2,1.00,1.0,0,0,3,6,770,0,1933,47.7379,-122.233,2720
3,4,2487200875,20141209,604000.0,4,3.00,1.0,0,0,5,7,1050,910,1965,47.5208,-122.393,1360
4,5,1954400510,20150218,510000.0,3,2.00,1.0,0,0,3,8,1680,0,1987,47.6168,-122.045,1800


In [45]:
from sklearn.model_selection import train_test_split

X_cols = data.columns.difference(['price', 'Unnamed: 0', 'id', 'date', 'waterfront', 'view', 'lat', 'long'])
X = data[X_cols]
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)
X.head()

,bathrooms,bedrooms,condition,floors,grade,sqft_above,sqft_basement,sqft_living15,yr_built
0,1.00,3,3,1.0,7,1180,0,1340,1955
1,2.25,3,3,2.0,7,2170,400,1690,1951
2,1.00,2,3,1.0,6,770,0,2720,1933
3,3.00,4,5,1.0,7,1050,910,1360,1965
4,2.00,3,3,1.0,8,1680,0,1800,1987


[('bathrooms', 'bedrooms', 'condition', 'floors', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15', 'yr_built'), ('bathrooms', 'bedrooms', 'condition', 'floors', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15'), ('bathrooms', 'bedrooms', 'condition', 'floors', 'grade', 'sqft_above', 'sqft_basement', 'yr_built'), ('bathrooms', 'bedrooms', 'condition', 'floors', 'grade', 'sqft_above', 'sqft_living15', 'yr_built'), ('bathrooms', 'bedrooms', 'condition', 'floors', 'grade', 'sqft_basement', 'sqft_living15', 'yr_built'), ('bathrooms', 'bedrooms', 'condition', 'floors', 'sqft_above', 'sqft_basement', 'sqft_living15', 'yr_built'), ('bathrooms', 'bedrooms', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15', 'yr_built'), ('bathrooms', 'bedrooms', 'floors', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15', 'yr_built'), ('bathrooms', 'condition', 'floors', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15', 'yr_built'), ('bedrooms', 'condition', 'floor

In [43]:
lassocv = sklearn.linear_model.LassoCV()
lassocv.fit(X, y)
lassocv_alpha = lassocv.alpha_
lassocv_score = lassocv.score(X, y)
print('LassoCV: alpha = {}, score = {}'.format(lassocv_alpha, lassocv_score))

print('MSE:', mse(lassocv.predict(X_test), y_test))
pd.DataFrame.from_dict({'feature': X.columns, 'weight': lassocv.coef_})

LassoCV: alpha = 190880.86219702454, score = 0.5337649016002417
MSE: 63159607934.52422


,feature,weight
0,bathrooms,0.000000
1,bedrooms,-0.000000
2,condition,0.000000
3,floors,0.000000
4,grade,0.000000
5,sqft_above,246.390987
6,sqft_basement,236.235964
7,sqft_living15,93.751322
8,yr_built,-2273.216848


In [63]:
def run_model(Model, xset, yset):
    inst = Model()
    inst.fit(xset, yset)
    return (inst.score(xset, yset),)#, inst.alpha_, inst.coef_

c = []
for r in range(len(X_cols), -1, -1):
    c += list(combinations(X_cols, r))


max([run_model(sklearn.linear_model.RidgeCV, data[list(selections)], y) + (selections,) for selections in c if selections])

(0.6216939248892939,
 ('bathrooms',
  'bedrooms',
  'condition',
  'floors',
  'grade',
  'sqft_above',
  'sqft_basement',
  'sqft_living15',
  'yr_built'))